#### Школа Менеджеров Яндекса

##### Продуктовая аналитика

###### Задача 1

Между офисами Яндекса в Москва-Сити, Красной Розе и Авроре курсируют шаттлы для сотрудников. Предположим, что есть 3 автобуса, по одному в каждом офисе. За день они делают только 2 рейса: в 9:00 и в 19:00. Вечерний рейс автобус совершает из того офиса, куда он приехал во время утреннего. Маршрут, по которому поедет дальше автобус, определяется в момент отправления исходя из количества заинтересованных пассажиров. В половине случаев в голосовании побеждают сотрудники одного офиса, в остальных случаях — другого. Даже если после утреннего рейса в каком-то офисе оказалось 2 шаттла, голосование в них проходит независимо друг от друга, то есть они оба могут поехать дальше по одному маршруту. Какова вероятность, что после вечернего рейса в каком-либо офисе окажутся два шаттла?

In [8]:
# изначальное расположение шаттлов
m = ['M', 'K', 'A']

# возможные расположения шаттлов после утренних рейсов
e = []
for i in m:
    m_cp = m.copy()
    m_cp.remove(i)
    for k in m_cp:
        e.append(i+k)

# возможные расположения шаттлов после вечерних рейсов
n = []
for i in e:
    m_cp = m.copy()
    m_cp.remove(i[-1])
    for k in m_cp:
        n.append(i+k)
        
# возможные расположения шаттлов после вечерних рейсов (только варианты два шаттла в одном офисе)
last_stop = []
for i in n:
    for k in n:
        if k[-1] == i[-1] and k != i:
            last_stop.append([i, k])
            break
            
# подсчет исходов и вероятности
all_outcomes = len(e) + len(n)
suitable_outcomes = len(last_stop)
p = suitable_outcomes / all_outcomes

# вывод результатов
print(f'''Число всех возможных исходов: {all_outcomes}
Число исходов два шаттла в одном офисе после вечернего рейса: {suitable_outcomes}
Вероятность два шаттла в одном офисе после вечернего рейса: {p:.2%}''')

Число всех возможных исходов: 18
Число исходов два шаттла в одном офисе после вечернего рейса: 12
Вероятность два шаттла в одном офисе после вечернего рейса: 66.67%


###### Задача 2

Яндекс запустил программу Плюс, которая позволяет копить кэшбек за пользование разными сервисами. Система работает как подписка, которая имеет срок действия, но может быть продлена.

Напишите SQL-запрос, чтобы посчитать количество пользователей, у которых доля покупок с активной подпиской составляет больше 80% от общего числа их заказов.

Структура таблиц:

market_orders

• order_id – ID заказа на Яндекс.Маркете

• utc_creation_dttm – дата создания заказа в формате YYYY-MM-DD hh:mm:ss

• user_id – ID пользователя

active_plus

• user_id – ID пользователя

• utc_dttm_from – дата в формате YYYY-MM-DD hh:mm:ss, начиная с которой подписка Плюс была активна

• utc_dttm_to – дата в формате YYYY-MM-DD hh:mm:ss, до которой подписка Плюс была активна

Гарантируется, что промежутки, когда подписка была активна, не пересекаются


In [9]:
# SQL запрос
WITH all_p AS 
(
SELECT
    user_id,
    CAST(COUNT (order_id) AS real) AS purchases_cnt
FROM market_orders
GROUP by user_id
), plus_p AS
(
SELECT
    m.user_id,
    CAST(COUNT (order_id) AS real) AS purchases_cnt
FROM market_orders AS M
LEFT JOIN active_plus AS a 
    ON m.user_id = a.user_id
WHERE m.utc_creation_dttm 
    BETWEEN a.utc_dttm_from AND a.utc_dttm_to
GROUP by m.user_id
)

SELECT
    COUNT(plus_p.user_id) AS users_cnt_above_80_percent
FROM plus_p
LEFT JOIN all_p
    ON plus_p.user_id = all_p.user_id
WHERE plus_p.purchases_cnt / all_p.purchases_cnt > 0.8

###### Задача 3

Напишите Python-функцию для проверки качества пароля в Яндекс.Почте. Хорошим считается пароль, который соответствует условиям:

    длина не менее 7 символов,
    содержит хотя бы 1 цифру,
    содержит хотя бы 1 букву верхнего регистра,
    содержит хотя бы 1 букву нижнего регистра,
    содержит только латинские буквы и цифры,
    не содержит повторяющиеся символы.



In [10]:
from string import ascii_letters
from string import digits


def password_checker(psw):
    symbols = list(psw)  # преобразование в список символов
    
    # серия проверок
    len_check = True if len(psw) > 7 else False  # проверка длины пароля
    digit = True if any([i.isdigit() for i in symbols]) else False  # проверка хотя бы 1 цифры
    upper = True if any([i.isupper() for i in symbols]) else False  # проверка буквы верхнего регистра
    lower = True if any([i.islower() for i in symbols]) else False  # проверка буквы нижнего регистра
    conditions = [len_check, digit, upper, lower]  # собираем результаты проверок
    
    # проверка по первым четырем условиям
    if not all(conditions):
        return 'Please, improve your password. You can do better!'
    
    # проверка на только латинские буквы и цифры И на повторяющиеся символы
    tmp = []
    for i in symbols:
        if i in ascii_letters or i in digits:  # проверка на только латинские буквы и цифры
            if i not in tmp:   # проверка на повторяющиеся символы
                tmp.append(i)
            else:
                return 'Please, improve your password. You can do better!'
        else:
            return 'Please, improve your password. You can do better!'
    
    # возвращаем сообщение, если пароль удовлетворяет всем условиям проверок
    return 'Congratulations! Your password looks great.'
            
    
password_checker('hj5789Jgf')

'Congratulations! Your password looks great.'

###### Задача 4

Коллеги Антон и Андрей живут в Москве и ездят на Такси. Антон пользуется тарифом UberX в Uber.Russia, а Андрей — тарифом Эконом в Яндекс.Go. Живут коллеги в соседних домах в 16,5 км от офиса. Каждый раз, когда они заказывают такси из офиса домой, цена их поездок оказывается разной: то у Антона дешевле, то у Андрея. Аналитик Саша подсказал, что, хотя сервисы являются продуктами одной компании, тарификация за минуту и километр в пути у них разная. Проверьте, как это влияет на стоимость поездки, и покажите, в каких случаях цена у Андрея выше, чем у Антона.
Допустим, что повышающего коэффициента не существует.

In [11]:
DIST = 16.5  # расстояние офис-дом

# расчитываем стоимость поездки для тарифов UBER
def uber_price(dist, speed):
    # тарифный план Uber
    UBER_BASIC_PRICE = 129
    UBER_MIN_INC = 5
    UBER_KM_INC = 4
    UBER_MIN_RUB = 6
    UBER_KM_RUB = 12
    
    # подсчет минут и км не включенных в базовую стоимость по тарифу
    uber_dist_payed = dist - UBER_KM_INC
    uber_min_payed = dist / (speed / 60) - UBER_MIN_INC
    
    # подсчет общей стоиомости поездки
    uber_total_price = UBER_BASIC_PRICE + uber_dist_payed * UBER_KM_RUB + uber_min_payed * UBER_MIN_RUB

    return round(uber_total_price, 2)


# расчитываем стоимость поездки для тарифов GO
def go_price(dist, speed):
    # тарифный план Go
    GO_BASIC_PRICE = 139
    GO_MIN_INC = 6
    GO_KM_INC = 3
    GO_MIN_RUB = 10
    GO_KM_RUB = 8

    # подсчет минут и км не включенных в базовую стоимость по тарифу
    go_dist_payed = dist - GO_KM_INC
    go_min_payed = dist / (speed / 60) - GO_MIN_INC

    # подсчет общей стоиомости поездки
    go_total_price = GO_BASIC_PRICE + go_dist_payed * GO_KM_RUB + go_min_payed * GO_MIN_RUB

    return round(go_total_price, 2)


# тестируем диапазон скоростей движения по городу (от 60 до 70 км/ч)
for velocity in range(60, 70):
    if uber_price(DIST, velocity) < go_price(DIST, velocity):
        print(f'''На дорогах пробки, средняя скорость движения по городу {velocity} км/ч.
Поездка на UBER дешевле на {round(abs(uber_price(DIST, velocity) - go_price(DIST, velocity)), 2)} рублей.\n''')
    else:
        print(f'''Город свободен, средняя скорость движения по городу {velocity} км/ч.
Поездка на Yandex Go выгоднее на {round(abs(uber_price(DIST, velocity) - go_price(DIST, velocity)), 2)} рублей.\n''')

На дорогах пробки, средняя скорость движения по городу 60 км/ч.
Поездка на UBER дешевле на 4.0 рублей.

На дорогах пробки, средняя скорость движения по городу 61 км/ч.
Поездка на UBER дешевле на 2.92 рублей.

На дорогах пробки, средняя скорость движения по городу 62 км/ч.
Поездка на UBER дешевле на 1.87 рублей.

На дорогах пробки, средняя скорость движения по городу 63 км/ч.
Поездка на UBER дешевле на 0.85 рублей.

Город свободен, средняя скорость движения по городу 64 км/ч.
Поездка на Yandex Go выгоднее на 0.12 рублей.

Город свободен, средняя скорость движения по городу 65 км/ч.
Поездка на Yandex Go выгоднее на 1.07 рублей.

Город свободен, средняя скорость движения по городу 66 км/ч.
Поездка на Yandex Go выгоднее на 2.0 рублей.

Город свободен, средняя скорость движения по городу 67 км/ч.
Поездка на Yandex Go выгоднее на 2.9 рублей.

Город свободен, средняя скорость движения по городу 68 км/ч.
Поездка на Yandex Go выгоднее на 3.76 рублей.

Город свободен, средняя скорость движения п

###### Задача 5

Яндекс.Музыка проводит офлайн-фестиваль самых популярных исполнителей жанра техно. Чтобы привлечь пользователей на мероприятие, маркетолог Иван должен разослать email пользователям Музыки. Отправка одного письма стоит 10 рублей, а цена билета составляет 700 рублей. Как бы вы посоветовали Ивану сформировать выборку для отправки, учитывая логи прослушиваний пользователей Яндекс.Музыки?

Обозначения в таблице:

    audition_id – ID прослушивания в логе

    utc_audition_start_dttm – дата начала прослушивания в формате YYYY-MM-DD hh:mm:ss

    utc_audition_end_dttm – дата окончания прослушивания в формате YYYY-MM-DD hh:mm:ss

    user_id – ID пользователя Яндекс.Музыки

    track_id – ID трека Яндекс.Музыки

    track_duration – продолжительность трека в секундах

    track_genre – жанр трека



In [12]:
# импорт библиотеки pandas
import pandas as pd

In [13]:
# загрузим логи прослушиваний из файла
df = pd.read_csv('music_data.zip', compression='zip', parse_dates=['utc_audition_end_dttm', 'utc_audition_start_dttm'])

In [14]:
# посчитаем и добавим в датафрейм длительность прослушивания в секундах
df['duration'] = (df.utc_audition_end_dttm - df.utc_audition_start_dttm).dt.total_seconds()

In [15]:
# отберем только треки жанра TECHNO
df_techno = df[df['track_genre'] == 'TECHNO']

In [16]:
# посчитаем общую длительность прослушивания треков жанра TECHNO с группировкой по пользователям
df_users = df_techno.groupby('user_id').agg({'duration': 'sum'})

In [17]:
# определим нижнюю границу выбросов значений (1.5*IQR, что соотетствует 2.698*sigma)
outliers_lower_boundary = df_users.duration.median() - 2.698 * df_users.duration.std()

In [18]:
# отберем пользователей выше 'нижней границы выбросов' значений длительности прослушивания
df_user_email = df_users.query('duration > @outliers_lower_boundary')

In [19]:
# cохраняем список ID пользователей Яндекс.Музыки для рассылки
user_ids_list = df_user_email.index.to_list()

In [20]:
# стоимости рассылки и билетов
ticket = 700
email = 10
conversion_rate = 0.05  # предположим конверсию 5% (уточнить у Ивана)
return_ = conversion_rate * len(user_ids_list) * ticket
invest = email * len(user_ids_list)
roi = (return_ - invest) / invest

# отчет
print(f'''Отобрали {len(user_ids_list)} пользователей для рассылки.
Стоимость рассылки составит {email * len(user_ids_list)} рублей.
При конверсии рассылки {conversion_rate:.2%} ROI составит {roi:.2%}.''')

Отобрали 1726 пользователей для рассылки.
Стоимость рассылки составит 17260 рублей.
При конверсии рассылки 5.00% ROI составит 250.00%.
